# 套件安裝

In [ ]:
# 安裝所需套件
!pip install -U openpyxl selenium beautifulsoup4 lxml requests

# 設定初始化

In [ ]:
'''
注意事項:
1. 程式執行前，需先連上臺師大 VPN。
2. 下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] 臺師大 圖書館 電子資料庫總覽
https://www.lib.ntnu.edu.tw/database/database_english.jsp?flag=0&choice_id=E
[2] EBSCO Select Resource
https://search.ebscohost.com/Community.aspx?community=y&authtype=ip
[3] python操作Excel檔案之openpyxl
https://www.itread01.com/content/1544850004.html
'''


'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 給 web driver 用的變數
driver = None

# 來源首頁
url = 'https://search.ebscohost.com/Community.aspx?community=y&authtype=ip'

# 檢索文字 (暫時以 list 型態定義檢索文字，未來有機會可以擴充使用)
listKeywords = [
    [
        'dissertation or thesis or "journal article" or "research paper"', 
        'plagiarism or cheating or "academic integrity"'
    ]
] 
fromYear = 1986
toYear = 2022

# 指定 sheet name
sheetName = '875'

# 指定 excel 檔名
excelFileName = 'EBSCO.xlsx'

# 指定 json 檔名
jsonFileName = f'{sheetName}.json'

# 建立儲存圖片、影片的資料夾
folderPath = f'./EBSCO_{sheetName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 判斷 excel 檔案是否存在，不存在就新增
filePath = folderPath + '/' + excelFileName
if not os.path.exists(filePath):
    workbook = Workbook() # 動態新增檔案
    worksheet = workbook.create_sheet(sheetName, 0) # 建立並取得 active sheet
else:
    workbook = load_workbook(filename = filePath)
    worksheet = workbook[sheetName] # 取得 active sheet

# 日期、來源、篇名、摘要、內文等標題
worksheet['A1'] = "檢索流水號"
worksheet['B1'] = "日期"
worksheet['C1'] = "來源"
worksheet['D1'] = "篇名"
worksheet['E1'] = "全文類型"
worksheet['F1'] = "固定連結"
worksheet['G1'] = "摘要"
worksheet['H1'] = "內文"

# 放置爬取的資料
listData = []

# 自訂函式 (網路爬蟲執行流程)

In [ ]:
'''
函式
'''
# 初始化 Web Driver
def init():
    global driver
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = driver_exec_path
    )

# 走訪來源網頁
def visit():
    global driver
    driver.get(url) #進入來源網頁
    try:
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.LINK_TEXT, 'EBSCOhost Web')
            )
        )
        
        # 按下超連結
        driver.find_element(By.LINK_TEXT, 'EBSCOhost Web').click()
    except TimeoutException as e:
        print('等待逾時: visit')

# 檢索
def search():
    global driver
    try:
        # 強制等待
        sleep( randint(1,2) )
        
        # 等待 選擇資料庫 文字出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'a#selectDBLink')
            )
        )
        
        # 按下 選擇資料庫
        driver.find_element(By.CSS_SELECTOR, 'a#selectDBLink').click()
        
        # 等待 選取/取消選取全部 checkbox 元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'input#selectAll')
            )
        )
        
        # 取得 選取/取消選取全部 checkbox 元素
        checkboxAllElement = driver.find_element(By.CSS_SELECTOR, 'input#selectAll')
        
        # 判斷 選取/取消選取全部 checkbox 元素是否勾選，勾選就取消
        if checkboxAllElement.is_selected():
            checkboxAllElement.click()
            
        # 強制等待
        sleep( randint(1,2) )
        
        # 取得指定的 checkbox 元素 (本例為 Newspaper Source)
        checkboxSpecificElement = driver.find_element(By.CSS_SELECTOR, 'input#ctrlSelectDb_dbList_ctl21_itemCheck')
        
        # 判斷 指定的 checkbox 元素是否勾選，沒勾選就直接勾選
        if not checkboxAllElement.is_selected():
            checkboxSpecificElement.click()
            
        # 強制等待
        sleep( randint(1,2) )
            
        # 按下 OK
        driver.find_element(By.CSS_SELECTOR, 'input#btnOK').click()
        
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'input#Searchbox1')
            )
        )
        
        # 點選(按鈕)選單元素 (本例為 TX)
        buttonDropDownElements = driver.find_elements(By.CSS_SELECTOR, 'button.dd-active')
        
        # 取得所有選單元素底下的列表
        ulDropDownElements = driver.find_elements(By.CSS_SELECTOR, 'ul.dd-list')
        
        # 輸入檢索文字，並選擇欄位
        inputSearchbox1 = driver.find_element(By.CSS_SELECTOR, 'input#Searchbox1')
        inputSearchbox1.send_keys(listKeywords[0][0])
        buttonDropDownElements[0].click()
        ulDropDownElements[0].find_elements(By.CSS_SELECTOR, 'li.dd-list-item')[1].click()
        
        # 輸入檢索文字，並選擇欄位
        inputSearchbox2 = driver.find_element(By.CSS_SELECTOR, 'input#Searchbox2')
        inputSearchbox2.send_keys(listKeywords[0][1])
        buttonDropDownElements[2].click()
        ulDropDownElements[2].find_elements(By.CSS_SELECTOR, 'li.dd-list-item')[1].click()
        
        # 取得 全文 checkbox 元素
        checkboxFullText = driver.find_element(By.CSS_SELECTOR, 'input#common_FT')
        
        # 判斷 全文 checkbox 元素是否勾選，沒勾選就直接勾選
        if not checkboxFullText.is_selected():
            checkboxFullText.click()
            
        # 輸入開始年份
        driver.find_element(By.CSS_SELECTOR, 'input#common_DT1_FromYear').send_keys(fromYear)
        
        # 輸入結束年份
        driver.find_element(By.CSS_SELECTOR, 'input#common_DT1_ToYear').send_keys(toYear)
        
        # 按下檢索
        driver.find_element(By.CSS_SELECTOR, 'input#ctl00_ctl00_MainContentArea_MainContentArea_ctrlLimiters_btnSearch').click()
        
    except TimeoutException as e:
        print('等待逾時: search')
        
# 取得主要連結與資訊
def getData():
    try:
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "a.arrow-link.legacy-link.next")
            )
        )
        
        # 判斷是否有下一頁，若有，則先蒐集資料，再按下一頁的連結
        while True:
            # 取得檢索結果(列表)
            li_elms = driver.find_elements(By.CSS_SELECTOR, "div#resultListControl.content-wrapper ul.result-list.has-icons > li.result-list-li")
            
            # 逐筆資料剖析
            for index, li in enumerate(li_elms):
                # 檢索流水號
                result_index = li.find_element(By.CSS_SELECTOR, 'span.record-index').get_attribute('innerText')
                result_index = result_index.replace(".", "")
                
                # 篇名 與 超連結
                a_elm = li.find_element(By.CSS_SELECTOR, 'div.result-list-record h3.title-link-wrapper a')
                str_title = a_elm.get_attribute('innerText')
                str_title_link = a_elm.get_attribute('href')
                
                # 來源 與 日期
                str_info = li.find_element(By.CSS_SELECTOR, 'div.result-list-record div.display-info').get_attribute('innerText')
                list_split_info = str_info.split("\n")
                str_info_source = list_split_info[2]
                regex_date = r'\d{1,2}\s*?\/\s*?\d{1,2}\s*?\/\s*?\d{2,4}'
                match_date = re.search(regex_date, str_info)
                str_date = match_date[0]
                
                # 全文類型
                a_elm = li.find_element(By.CSS_SELECTOR, 'div.result-list-record div.display-info span.record-formats a')
                str_fulltext_type = a_elm.get_attribute('innerText')
                regexType = r'[A-Z]+'
                matchType = re.search(regexType, str_fulltext_type)
                str_fulltext_type = matchType[0]
                
                # 開啟新分頁
                driver.execute_script(f'window.open("{str_title_link}", "_blank");')

                # 切換到分頁
                driver.switch_to.window(driver.window_handles[1])
                
                # 等描述資料相關元素出現
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "dl#citationFields")
                    )
                )
                
                # 取得描述資料區塊的元素
                dt_elms = driver.find_elements(By.CSS_SELECTOR, 'dl#citationFields dt[data-auto="citation_field_label"]')
                dd_elms = driver.find_elements(By.CSS_SELECTOR, 'dl#citationFields dd[data-auto="citation_field_value"]')
                
                # 摘要、內文、全文連結、pdf 下載連結 等預設值
                str_abstract = str_inner_text = str_permalink = str_pdf_link = ""
                
                if str_fulltext_type == 'HTML': # HTML 取得方式
                    # 摘要
                    for index, dt in enumerate(dt_elms):
                        if "摘要" in dt.get_attribute('innerText'):
                            str_abstract = dd_elms[index].get_attribute('innerText')
                            break

                    # 內文
                    children_in_section_elms = driver.find_elements(By.CSS_SELECTOR, 'section#TextToSpeech > *')
                    for index, elm in enumerate(children_in_section_elms):
                        if elm.get_attribute('data-auto') == 'copyright_info': continue
                        if elm.get_attribute('id') == 'textToSpeechPlaceholder': continue
                        if elm.get_attribute('data-auto') == 'fulltext_title_hidden': continue
                        str_inner_text += elm.get_attribute('innerText') + " \n"
                    
                elif str_fulltext_type == 'PDF': # PDF 取得方式
                    # 判斷頁面是否有 pdf 全文連結，如果找不到，則刷新頁面試試
                    for i in range(10):
                        if len( driver.find_elements(By.CSS_SELECTOR, 'a.record-type.pdf-ft') ) > 0 and driver.find_element(By.CSS_SELECTOR, 'a.record-type.pdf-ft').is_displayed():
                            break
                        else:
                            driver.refresh()
                            sleep( randint(1,2) )
                    
                    # 進入 pdf viewer 頁面
                    driver.find_element(By.CSS_SELECTOR, 'a.record-type.pdf-ft').click()
                    
                    # 等待 pdf viewer 元素出現
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'input#pdfUrl')
                        )
                    )
                    
                    # 取得 pdf 連結
                    str_pdf_link = driver.find_element(By.CSS_SELECTOR, 'input#pdfUrl').get_attribute('value')
                    
                    # 下載 pdf
                    cmd = ['curl', str_pdf_link, '-o', f'{folderPath}/{sheetName}-{result_index}.pdf']
                    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
                    output = result.stdout
                    print("=" * 20)
                    print(f"{sheetName}-{result_index}.pdf 下載完成")
                    print("=" * 20)
                
                # 等待
                sleep(1)
                
                # 如果找不到 固定連結，則刷新頁面試試
                for i in range(10):
                    if len( driver.find_elements(By.CSS_SELECTOR, 'ul.article-tools.delivery-control > li.article-tool > a.permalink-link') ) > 0 and driver.find_element(By.CSS_SELECTOR, 'ul.article-tools.delivery-control > li.article-tool > a.permalink-link').is_displayed():
                        break
                    else:
                        driver.refresh()
                        sleep( randint(1,2) )

                # 如果還是找不到 固定連結，則以當前連結為 固定連結
                if driver.find_element(By.CSS_SELECTOR, 'ul.article-tools.delivery-control > li.article-tool > a.permalink-link').is_displayed():
                    # 按下 固定連結 圖示
                    driver.find_element(By.CSS_SELECTOR, 'ul.article-tools.delivery-control > li.article-tool > a.permalink-link').click()

                    # 等待 固定連結 元素出現
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, 'input#pLinkDetail')
                        )
                    )
                    
                    # 強制等待
                    sleep( randint(1,2) )

                    # 取得 固定連結
                    str_permalink = driver.find_element(By.CSS_SELECTOR, 'input#pLinkDetail').get_attribute('value')        
                else:
                    # 依然找不到，則取得當前連結，作為 固定連結
                    str_permalink = driver.current_url
                        
                # 關閉當前分頁
                driver.close()
                
                # 切換到初始分頁
                driver.switch_to.window(driver.window_handles[0])
                
                # 整理資料
                listData.append({
                    "檢索流水號": result_index,
                    "日期": str_date,
                    "來源": str_info_source,
                    "篇名": str_title,
                    "全文類型": str_fulltext_type,
                    "固定連結": str_permalink,
                    "摘要": str_abstract,
                    "內文": str_inner_text
                })
                
                print(f"已走訪編號: {result_index}")
                
            # 按 下一頁(個)，如果沒有下一頁，則跳出迴圈
            if len( driver.find_elements(By.CSS_SELECTOR, "a.arrow-link.legacy-link.next") ) > 0:
                driver.find_element(By.CSS_SELECTOR, "a.arrow-link.legacy-link.next").click()
            else:
                break
            
    except TimeoutException as e:
        print("等待逾時: getMainData")

# 關閉瀏覽器
def close():
    driver.quit()
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )
        
# 儲存成 excel
def saveExcel():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:
        # 從 excel 列號 2 開始寫入資料
        row_num = 2
        
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
        # 逐列寫入
        for myDict in listJson:
            worksheet['A' + str(row_num)] = myDict['檢索流水號']
            worksheet['B' + str(row_num)] = myDict['日期']
            worksheet['C' + str(row_num)] = myDict['來源']
            worksheet['D' + str(row_num)] = myDict['篇名']
            worksheet['E' + str(row_num)] = myDict['全文類型']
            worksheet['F' + str(row_num)] = myDict['固定連結']
            worksheet['G' + str(row_num)] = myDict['摘要']
            worksheet['H' + str(row_num)] = myDict['內文']
            row_num += 1
    
    # 儲存 workbook
    workbook.save(filePath)

    # 關閉 workbook
    workbook.close()

# 注意
- 以下的函數必須各別執行。
- 若是中間需要透過瀏覽器讀取網頁後，一定要等瀏覽器讀取完，才能往下一個流程執行。
- 只要瀏覽器沒關閉(close)，中間的檢索(search)與取得主要資訊(getData)都可以修改程式後，重新依序執行。
- 只要建立完 json 檔 (saveJson)，便可建立 excel 檔 (saveExcel)，與網路爬蟲流程是分開的。
- 這個網站麻煩的地方在於，**它提供的超連結，只限此時開啟瀏覽器期間可用，若是關閉瀏覽器重開，網站的 token 一換，舊的超連結就無法使用了**。

In [ ]:
# 開啟 Web Driver
init()

In [ ]:
# 走訪來源網頁 (有時候瀏覽器會執行比較久，要等它讀完網頁，才能執行下一個流程 search)
visit()

In [ ]:
# 檢索 (有時候瀏覽器會執行比較久，要等它讀完網頁，才能執行下一個流程 getData)
search()

In [ ]:
# 取得主要連結與資訊 (記得要先等上一步 search 流程的瀏覽器先讀完，不然會有問題)
time_begin = time.time()
getData()
time_end = time.time()
print(f"總共執行了 { time_end - time_begin } 秒")

In [ ]:
# 手動關閉瀏覽器
close()

In [ ]:
# 將 listData 儲存成 json
saveJson()

In [ ]:
# 將 json 資料儲存至 excel 當中 (要先有 json 檔)
saveExcel()